In [ ]:
"""
Removing particles from the LB3D checkpoint file
================================================

Opening an LB3D checkpoint and removing particles to the current simulation
based on particle positions added with fibonacci's method.
"""

import numpy as np
import glob
from schiller_lab_tools.lb3d import checkpoint_io
from schiller_lab_tools.data import particles
import copy
import os

In [ ]:
## EDIT THIS ONLY ##
path = "../../tests/data/lb3d"
nx, ny, nz = [64, 64, 128]
Q = 19
timestep = 10000
nprocs = 4
Rp = 8
Ro = 4
npart = 10
rhof = 0.7
radius_prop = 0.7 # assumes z is the long axis if box size is not cubic

gr_out = "concatenated"
npart_new = 25
join_cut = 30
other_cut = 20
## EDIT THIS ONLY ##

output_path = f"{path}/concat_system_slice/center_distance_{nz-2*join_cut:d}"
os.makedirs(output_path, exist_ok=True)

In [2]:
path = "anisotropic_particle"
nx, ny, nz = [64, 64, 128]
Q = 19
timestep = 10000
nprocs = 4
Rp = 8
Ro = 4
npart_old = 5
rhof = 0.7
radius_prop = 0.7 # assumes z is the long axis if box size is not cubic

gr_out = "restart"
## EDIT THIS ONLY ##

output_path = f"{path}/remove_particles/"
os.makedirs(output_path, exist_ok=True)

In [3]:
## READING OLD CHECKPOINTS ##
checkparams_files = sorted(glob.glob(f"{path}/checkparams*{timestep:08d}*.xdr"))[0]
fluid_checkpoint_files = sorted(glob.glob(f"{path}/checkpoint*{timestep:08d}*.xdr"))
checktopo_files = sorted(glob.glob(f"{path}/checktopo*{timestep:08d}*.xdr"))[0]
md_checkpoint_files = sorted(glob.glob(f"{path}/md-checkpoint*{timestep:08d}*.xdr"))[0]

curr_check_params = read_checkparams_xdr(checkparams_files)
curr_topo = read_checktopo_xdr(checktopo_files, nprocs = nprocs)
curr_fluid_params = read_checkpoint_xdr(fluid_checkpoint_files, nx, ny, nz, curr_topo, Q)
curr_md_params = read_md_checkpoint_xdr(md_checkpoint_files, use_rotation=True, interaction="ladd", n_spec = 2)
print("Checkpoints to be copied have been read")
## READING OLD CHECKPOINTS ##

Checkpoints to be copied have been read


# Parameter file

In [4]:
## PARAMETER CHECKPOINT FILE ##
new_check_params = copy.deepcopy(curr_check_params)
print("New parameter file generated")
## PARAMETER CHECKPOINT FILE ##

New parameter file generated


# Fluid files

In [5]:
## FLUID CHECKPOINT FILE ##
weights = np.array([1./3.,
                    1./18.,
                    1./18.,
                    1./18.,
                    1./18.,
                    1./18.,
                    1./18.,
                    1./36.,
                    1./36.,
                    1./36.,
                    1./36.,
                    1./36.,
                    1./36.,
                    1./36.,
                    1./36.,
                    1./36.,
                    1./36.,
                    1./36.,
                    1./36.
                    ])

od = np.sum(curr_fluid_params[..., :Q], axis = -1)
od = fill(od)
od = np.einsum("ijk, l -> ijkl", od, weights)

wd = np.sum(curr_fluid_params[..., Q:2*Q], axis = -1)
wd = fill(wd)
wd = np.einsum("ijk, l -> ijkl", wd, weights)

new_fluid_params = curr_fluid_params.copy()
new_fluid_params[..., :Q] = od
new_fluid_params[..., Q:2*Q] = wd
## FLUID CHECKPOINT FILE ##

# Topology

In [6]:
## TOPOLOGY CHECKPOINT FILE ##
new_topo = copy.deepcopy(curr_topo)
new_topo['cdims'] = [1,1,1]
new_topo['all_ccoords'] = [0,0,0]
new_topo['nprocs'] = 1
print("New mpi topology file generated")
## TOPOLOGY CHECKPOINT FILE ##

New mpi topology file generated


# MD Files

In [7]:
## MD CHECKPOINT FILE ##
new_md_params = copy.deepcopy(curr_md_params)

new_particles = [] 

## mass correction scheme also needs to be adjusted. 
ladd_data = new_md_params['ladd_data']
new_ladd_data = copy.deepcopy(ladd_data)
new_target_mass = np.array([np.sum(od)/new_check_params['taubulk_r'], np.sum(wd)/new_check_params['taubulk_b']])
new_ladd_data["global_mass_target"] = new_target_mass

new_md_params['particles'] = new_particles
new_md_params['ladd_data'] = new_ladd_data
print("New MD checkpoint file generated")
## MD CHECKPOINT FILE ##

New MD checkpoint file generated


# Output

In [8]:
## OUTPUTTING NEW CHECKPOINTS ##
uid = np.random.randint(0, 2**31, 1)[0] # Generating random number of a signed FP32 integer

checkparams_file_template = "checkparams_{0}_t{1:08d}-{2:010d}.xdr"
fluid_checkpoint_file_template = "checkpoint_{0}_t{1:08d}-{2:010d}_p{3:06d}.xdr"
checktopo_file_template = "checktopo_{0}_t{1:08d}-{2:010d}.xdr"
md_checkpoint_file_template = "md-checkpoint_{0}_t{1:08d}-{2:010d}.xdr"

output_params_path = output_path + "/" + checkparams_file_template.format(gr_out, timestep, uid)
output_fluid_path = output_path + "/" + fluid_checkpoint_file_template.format(gr_out, timestep, uid, 0)
output_topo_path = output_path + "/" + checktopo_file_template.format(gr_out, timestep, uid)
output_md_check_path = output_path + "/" + md_checkpoint_file_template.format(gr_out, timestep, uid)

write_checkparams_xdr(output_params_path, new_check_params)
write_checkpoint_xdr(output_fluid_path, new_fluid_params, nx, ny, nz)
write_checktopo_xdr(output_topo_path, new_topo)
write_md_checkpoint_xdr(output_md_check_path, new_md_params["particles"], 
                        use_rotation=True, steps_per_lbe_step=1, interaction="ladd",
                        ladd_props=new_md_params['ladd_data'],
                        n_spec=2)

print(f"Checkpoint output successful!. UID:{uid}")
## OUTPUTTING NEW CHECKPOINTS ##

Checkpoint output successful!. UID:1644876659
